# Fine-tuning pre-processing
-----

En este código de ejemplo se van a tratar los siguientes aspectos relevantes para el fine-tunning en este orden:


1. **Preparación de los datos**
    - **Carga de los datos**
    - **Reformateo para que sigan el formato indicado más adelante**
2. **Validación de los datos y detección de errores en ellos**
    - **Busqueda de errores**
    - **Estudio de la distribución de los mensajes**
3. **Estimación del coste del entrenamiento**
4. **Almacenamiento de la BD**
5. **Subida de BD a OpenAI para su posterior uso**
    - **Crear un trabajo de fine-tunning**
6. **Como usar un modelo de Fine-tunning**



In [4]:
import os
os.environ['OPENAI_API_KEY'] = 'Fill'

In [6]:
import openai
import pickle

In [7]:
pkl_saved = "../../data/Conversations/HTTP/pickle/HTTP_Conv2_validation_pairs_Aday.pkl"
file_path = 'DBs/HTTP/Conversations/HTTP_validation.jsonl'

system_message = 'You are a new generation traffic generator. \
You are specilized in the HTTP protocol and conversations generation using python and scapy. \
You are especially attentive to variables and different types of traffic.'

## **Step 1: Prepare your data**

### 1.1 Cargar datos

Cargamos los diálogos.

In [13]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

responses = pickle.load( open(pkl_saved, "rb" ))

In [15]:
data = []

for item in responses:
    data.append({"prompt":item.prompt_summary,"completion":item.completion.choices[0].text})
    #data[-1]["prompt"] += "\n###\n\n"
    #data[-1]["completion"] = "\n" + data[-1]["completion"]
    #data[-1]["completion"] += "\n\n###"

print(data[-1])

{'prompt': 'Source: IP="104.16.41.2", port=5016, Window: 17000 // Destination: IP="104.40.211.35", port=80, Window: 8500 // Others: Host="http://www.bluehost.com/", Path="", Code=307, Reason_Phrase= "Temporary Redirect"\n', 'completion': '\nx,y = random.sample(range(1, 100), 2)\npkt_1= IP(src="104.16.41.2", dst="104.40.211.35")/TCP(sport=5016, dport=80, flags="S", seq=y, window=17000)\ntime.sleep(abs(random.gauss(0, 0.03)))\ny += len(pkt_1[TCP].payload)+1\npkt_2= IP(src="104.40.211.35", dst="104.16.41.2")/TCP(sport=80, dport=5016, flags="SA", seq=x, ack=y, window=8500)\ntime.sleep(abs(random.gauss(0, 0.03)))\nx += len(pkt_2[TCP].payload)+1\npkt_3= IP(src="104.16.41.2", dst="104.40.211.35")/TCP(sport=5016, dport=80, flags="A", seq=y, ack=x, window=17000)\ntime.sleep(abs(random.gauss(0, 0.03)))\ny += len(pkt_3[TCP].payload)\npkt_4= IP(src="104.16.41.2", dst="104.40.211.35")/TCP(sport=5016, dport=80, flags="PA", seq=y, ack=x, window=17000)/HTTP()/HTTPRequest(Method="GET", Http_Version="HT

##### Función para dividir prompts múltiples en parejas resumen-comando, haciendo que todas las relaciones sean 1 a 1 y no N a N

In [374]:
def Todoen1(data):
    DB= []
    for conjunto in data:
        lines_query =[]
        lines_completion = []
        #print("-------------------------------------------------------------------------------------")
        #print(conjunto)
        lines_query = conjunto["prompt"].splitlines()
        #print("-------------------------------------------------------------------------------------")
        #print(lines_query)
        lines_completion = str(conjunto["completion"])
        #print(lines_completion)
        #lines_completion = lines_completion[1:]
        #print("-------------------------------------------------------------------------------------")
        #print(lines_completion)
        #print("-------------------------------------------------------------------------------------")
        #print("-------------------------------------------------------------------------------------")
        #print(len(lines_query))
        for index in range(len(conjunto)-1):
            #print(lines_query[index])
            #print(lines_completion[index][:-1])
            #print("-------------------------------------------------------------------------------------")
            #print("-------------------------------------------------------------------------------------")
            DB.append({"prompt":lines_query[index],"completion":lines_completion})
        
    return DB

##### Función para procesar prompts y dejarlos como están de manera que relación sea N a N

In [377]:
def TodoTalCual(data):
    DB= []
    for conjunto in data:
        lines_query =[]
        lines_completion = []
        #print("-------------------------------------------------------------------------------------")
        #print(conjunto)
        lines_query = conjunto["prompt"].splitlines()
        #print("-------------------------------------------------------------------------------------")
        #print(str(lines_query))
        lines_completion = conjunto["completion"].splitlines()
        #lines_completion = lines_completion[2:-2]
        #print("-------------------------------------------------------------------------------------")
        #print(lines_completion)
        print("-------------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------------")
        #print(len(lines_query))
        completion =[]
        for index in range(len(lines_completion)):
            completion.append(lines_completion[index][:-1].strip())
        #print(completion)
        #print("-----")
        DB.append({"prompt":lines_query[index],"completion":str(completion)})
        
    return DB

### 1.2 Aplicar formato necesario

Tratamiento de los datos de entrada en función de la salida que se quiera

In [380]:
# Si se quiere parejas de 1 a 1
DB = Todoen1(data)
# Si se quiere relaciones N a N
#DB = TodoTalCual(data)

In [382]:
print(DB[9])

{'prompt': 'Source: IP="192.229.162.112", port=5013, Window: 2000 // Destination: IP="13.77.161.179", port=80, Window: 11500 // Others: Host="http://www.edgecast.com/", Path="", Code=307, Reason_Phrase= "Temporary Redirect"', 'completion': '\nx,y = random.sample(range(1, 100), 2)\npkt_1= IP(src="192.229.162.112", dst="13.77.161.179")/TCP(sport=5013, dport=80, flags="S", seq=y, window=2000)\ntime.sleep(abs(random.gauss(0, 0.03)))\ny += len(pkt_1[TCP].payload)+1\npkt_2= IP(src="13.77.161.179", dst="192.229.162.112")/TCP(sport=80, dport=5013, flags="SA", seq=x, ack=y, window=11500)\ntime.sleep(abs(random.gauss(0, 0.03)))\nx += len(pkt_2[TCP].payload)+1\npkt_3= IP(src="192.229.162.112", dst="13.77.161.179")/TCP(sport=5013, dport=80, flags="A", seq=y, ack=x, window=2000)\ntime.sleep(abs(random.gauss(0, 0.03)))\ny += len(pkt_3[TCP].payload)\npkt_4= IP(src="192.229.162.112", dst="13.77.161.179")/TCP(sport=5013, dport=80, flags="PA", seq=y, ack=x, window=8192)/HTTP()/HTTPRequest(Method="GET", 

### 1.3 Aplicar formato necesario
Ahora debemos asegurarnos que cada ejemplo siga el siguiente formato:

```
{
  "messages": [
    { "role": "system", "content": "You are an assistant that occasionally misspells words" },
    { "role": "user", "content": "Tell me a story." },
    { "role": "assistant", "content": "One day a student went to schoool." }
  ]
}
```

Vamos a programar una función que construye cada ejemplo como un diccionario con una única llave `messages` y cuyo valor es el mensaje del sistema, más la conversación entre usuario y asistente.

In [386]:
def formatear_datos(data, system_message=None):
    messages = []

    # Incluir primero el mensaje de sistema
    if system_message:
        messages.append({
            "role": "system",
            "content": system_message
        })
    # Iterar por la lista de mensajes
    user_message = {
        "role": "user",
        "content": data["prompt"]
    }

    #Agregar el mensaje a la lista
    messages.append(user_message)
    
    assistant_message = {
        "role":"assistant",
        "content": data["completion"]
    }
    
    #Agregar el mensaje a la lista
    messages.append(assistant_message)

    # Crear diccionario final
    dict_final = {
        "messages": messages
    }

    return dict_final

Aplicamos la función a cada ejemplo.

In [389]:
dataset = []

conjunto = []
for conjunto in DB:
    ejemplo_formateado = formatear_datos(data=conjunto,
                                        system_message=system_message)

    dataset.append(ejemplo_formateado)
    conjunto = []

In [391]:
print(dataset[0])

{'messages': [{'role': 'system', 'content': 'You are a new generation traffic generator. You are specilized in the HTTP protocol and conversations generation using python and scapy. You are especially attentive to variables and different types of traffic.'}, {'role': 'user', 'content': 'Source: IP="199.232.37.140", port=5010, Window: 2900 // Destination: IP="13.107.246.10", port=80, Window: 14500 // Others: Host="http://www.maxcdn.com/", Path="", Code=307, Reason_Phrase= "Temporary Redirect"'}, {'role': 'assistant', 'content': '\nx,y = random.sample(range(1, 100), 2)\npkt_1= IP(src="199.232.37.140", dst="13.107.246.10")/TCP(sport=5010, dport=80, flags="S", seq=y, window=2900)\ntime.sleep(abs(random.gauss(0, 0.03)))\ny += len(pkt_1[TCP].payload)+1\npkt_2= IP(src="13.107.246.10", dst="199.232.37.140")/TCP(sport=80, dport=5010, flags="SA", seq=x, ack=y, window=14500)\ntime.sleep(abs(random.gauss(0, 0.03)))\nx += len(pkt_2[TCP].payload)+1\npkt_3= IP(src="199.232.37.140", dst="13.107.246.10

## **Step 2: Validar formato, errores, y distribuciones**

Revisamos si hay errores y estimamos el precio usando la guía [entregada por OpenAI](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset)

### 2.1 Busqueda de errores

In [396]:
# Format error checks
from collections import defaultdict
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


### 2.2 Estudio Distribuciones

In [399]:
import tiktoken
import numpy as np
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribución de {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"media / mediana: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")


In [401]:
# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num de ejemplos sin el system message:", n_missing_system)
print("Num de ejemplos sin el user message:", n_missing_user)
print_distribution(n_messages, "num_mensajes_por_ejemplo")
print_distribution(convo_lens, "num_total_tokens_por_ejemplo")
print_distribution(assistant_message_lens, "num_assistant_tokens_por_ejemplo")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} ejemplos que excedan el límite de tokenes de 4096, ellos serán truncados durante el fine-tuning")


Num de ejemplos sin el system message: 0
Num de ejemplos sin el user message: 0

#### Distribución de num_mensajes_por_ejemplo:
min / max: 3, 3
media / mediana: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribución de num_total_tokens_por_ejemplo:
min / max: 712, 1056
media / mediana: 906.0526315789474, 1032.0
p5 / p95: 714.0, 1054.0

#### Distribución de num_assistant_tokens_por_ejemplo:
min / max: 589, 931
media / mediana: 782.3157894736842, 910.0
p5 / p95: 590.0, 930.0

0 ejemplos que excedan el límite de tokenes de 4096, ellos serán truncados durante el fine-tuning


## **Step 3: Estimación del coste**

In [404]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 4
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"El conjunto de datos tiene ~{n_billing_tokens_in_dataset} tokens que serán cargados durante el entrenamiento")
print(f"Por defecto, entrenarás para {n_epochs} epochs en este conjunto de datos")
print(f"Por defecto, serás cargado con ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("Revisa la página para estimar el costo total")

El conjunto de datos tiene ~17215 tokens que serán cargados durante el entrenamiento
Por defecto, entrenarás para 5 epochs en este conjunto de datos
Por defecto, serás cargado con ~86075 tokens
Revisa la página para estimar el costo total


## **Step 4: Guardar datos fromateados**

Guardamos la base de datos en JSONL=JSON Lines.

In [408]:
import json

def save_to_jsonl(dataset, file_path):
    with open(file_path, 'w') as file:
        for ejemplo in dataset:
            json_line = json.dumps(ejemplo, ensure_ascii=False)
            file.write(json_line + '\n')

In [410]:
#Guardar train full
save_to_jsonl(dataset, file_path)

## **Step 5: Upload files**

Cargamos la base de datos a OpenAI y luego imprimimos el id de la respuesta de esta solicitd. Hacemos esto porque vamos a necesitar el id posteriormente.

In [214]:
from openai import OpenAI
openai.api_key = 'Fill'
client = OpenAI()

client.files.create(
  file=open(file_path, "rb"),
  purpose="fine-tune"
)

FileObject(id='file-LeipWBmVD1vReGFB3F1eahRq', bytes=17405, created_at=1715619052, filename='ARP_DB_validation_v2.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

### 5.1 Create a fine-tuning job

Luego creamos un punto de trabajo para hacer fine-tuning.

In [83]:
protocol = "ARPv2"

import json

with open('DBs/OpenAI_DBs_IDs.jsonl', 'r') as json_file:
    IDs_list = json.load(json_file)

print(IDs_list["Training"][protocol])

file-J22nrKS4x4DfCLRxTdJbNO2m


In [89]:
response = client.fine_tuning.jobs.create(training_file=IDs_list["Training"][protocol],
                                          model="gpt-3.5-turbo-1106", 
                                          suffix=protocol,
                                          hyperparameters={'n_epochs':4},
                                          validation_file=IDs_list["Validation"][protocol])

In [86]:
response

FineTuningJob(id='ftjob-ZdZGVl34NgHQSlzGXz0UzSwY', created_at=1713459610, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=4, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-ULHsc1YYFDmMSDkN1nMjJgTG', result_files=[], status='validating_files', trained_tokens=None, training_file='file-J22nrKS4x4DfCLRxTdJbNO2m', validation_file='file-M7hHyuyUJcmGX9v1gRAqjoQS', user_provided_suffix='ARPv1', seed=1520269638, integrations=[])

In [142]:
openai.FineTuningJob.retrieve(response.id)

APIRemovedInV1: 

You tried to access openai.FineTuningJob, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
response = openai.FineTuningJob.list_events(id=response.id)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


## **Step 6: Use a fine-tuned model**

Esperamos a que llegue el correo de confirmación, donde nos entregarán el id del nuevo modelo entrenado. Usaremos langchain (revisa aquí el último tutorial).

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

model_name = "ft:gpt-3.5-turbo-0613:evo-academy:burro-shrek:7tg5aZZV"
chat = ChatOpenAI(model=model_name, temperature=0.0)

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content="Hola! Soy Jonathan, tanto tiempo que no hablamos. Qué tal tu día?")
]

response = chat(messages)
print(response.content)

In [ ]:
chat = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content="Hola! Soy Jonathan, tanto tiempo que no hablamos. Qué tal tu día?")
]

response = chat(messages)
print(response.content)

¡Hola Jonathan! ¡Mucho gusto verte de nuevo! Mi día ha sido bastante interesante, he estado aquí, charlando y respondiendo preguntas. ¿Y tú, cómo ha sido tu día? [levanto una oreja con curiosidad]


Síguenos en nuestras redes:
- TikTok: https://www.tiktok.com/@evoacdm
- Instagram: https://www.instagram.com/evoacdm/
- LinkedIn: https://www.linkedin.com/company/evoacmd/